# Network analysis

Neurons do not act in isolation -- their pairwise interactions form a
network whose topology reveals modular organization and hub neurons.
[**DRIADA**](https://driada.readthedocs.io) builds these networks from
significance-tested mutual information and provides both structural and
spectral analysis tools.

| Step | Notebook | What it does |
|---|---|---|
| **Overview** | [00 -- DRIADA overview](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/00_driada_overview.ipynb) | Core data structures, quick tour of INTENSE, DR, networks |
| Load & inspect | [01 -- Data loading](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/01_data_loading_and_neurons.ipynb) | Wrap your recording into an `Experiment`, reconstruct spikes, assess quality |
| Single-neuron selectivity | [02 -- INTENSE](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/02_selectivity_detection_intense.ipynb) | Detect which neurons encode which behavioral variables |
| Population geometry | [03 -- Dimensionality reduction](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/03_population_geometry_dr.ipynb) | Extract low-dimensional manifolds from population activity |
| **Network analysis** | **04 -- this notebook** | Build and analyze cell-cell interaction graphs |
| Putting it together | [05 -- Advanced](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/05_advanced_capabilities.ipynb) | Combine INTENSE + DR, leave-one-out importance, RSA, RNN analysis |

**Overview:** INTENSE computes pairwise mutual-information between
all neuron pairs and determines which connections are statistically
significant via permutation testing. The resulting adjacency matrix
defines a *functional network* whose topology reveals modular
organization, hub neurons, and spectral fingerprints.

**Sections:**

1. **Network structure** -- Binary and weighted network properties,
   module detection (Louvain), degree distribution, null model
   comparison (degree-preserving randomization).
2. **Spectral analysis** -- Eigendecomposition of adjacency and
   normalized Laplacian matrices. IPR (eigenvector localization),
   complex spacing ratios, localization signatures, thermodynamic
   entropy, communicability, hyperbolicity, Laplacian Eigenmaps
   embedding, and directed-network eigenvalue analysis.

In [ ]:
# TODO: revert to '!pip install -q driada' after v1.0.0 PyPI release
!pip install -q git+https://github.com/iabs-neuro/driada.git@main
%matplotlib inline

import tempfile

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.community as nx_comm
import scipy.sparse as sp
from scipy.stats import gaussian_kde

from driada.network import Network
from driada.network.matrix_utils import turn_to_partially_directed
from driada.intense import compute_cell_cell_significance
from driada.experiment.synthetic import generate_tuned_selectivity_exp

## Building the network

Create a synthetic modular population (120 neurons, 6 functional
groups) with [`generate_tuned_selectivity_exp`](https://driada.readthedocs.io/en/latest/api/experiment/synthetic.html#driada.experiment.synthetic.generators.generate_tuned_selectivity_exp),
compute pairwise significance with
[`compute_cell_cell_significance`](https://driada.readthedocs.io/en/latest/api/intense/pipelines.html#driada.intense.pipelines.compute_cell_cell_significance),
and construct a [`Network`](https://driada.readthedocs.io/en/latest/api/network/core.html#driada.network.net_base.Network)
object.  This network is reused in both analysis sections below.

In [ ]:
def create_modular_experiment(duration=300, seed=42):
    """
    Create synthetic experiment with hierarchical modular structure.

    Creates 120 neurons in 6 functional groups:
    - 3 single-feature modules (30 neurons each): respond to event_0, event_1, or event_2
    - 3 dual-feature modules (10 neurons each): respond to pairs of events in OR mode
      (event_0 OR event_1, event_0 OR event_2, event_1 OR event_2)

    This creates a realistic hierarchical network with both specialized
    and multi-selective neurons.
    """
    # Create population with mixed selectivity
    population = [
        # Single-feature modules (30 neurons each)
        {
            "name": "event_0_cells",
            "count": 30,
            "features": ["event_0"],
        },
        {
            "name": "event_1_cells",
            "count": 30,
            "features": ["event_1"],
        },
        {
            "name": "event_2_cells",
            "count": 30,
            "features": ["event_2"],
        },
        # Dual-feature modules (10 neurons each, OR combination)
        {
            "name": "event_0_or_1_cells",
            "count": 10,
            "features": ["event_0", "event_1"],
            "combination": "or",
        },
        {
            "name": "event_0_or_2_cells",
            "count": 10,
            "features": ["event_0", "event_2"],
            "combination": "or",
        },
        {
            "name": "event_1_or_2_cells",
            "count": 10,
            "features": ["event_1", "event_2"],
            "combination": "or",
        },
    ]

    # Generate experiment with hierarchical structure
    exp = generate_tuned_selectivity_exp(
        population=population,
        n_discrete_features=3,  # Three distinct events
        duration=duration,
        fps=20.0,
        baseline_rate=0.05,
        peak_rate=2.0,
        decay_time=2.0,
        calcium_noise=0.02,
        seed=seed,
        verbose=True
    )

    # Return info about true module structure
    n_modules = 6
    module_sizes = [30, 30, 30, 10, 10, 10]

    return exp, n_modules, module_sizes


print("Creating synthetic experiment with hierarchical modular structure...")
print("  120 neurons: 30+30+30 (single-feature) + 10+10+10 (dual-feature)")
exp, n_modules_true, module_sizes_true = create_modular_experiment(duration=300)
print(f"Created {len(exp.neurons)} neurons in {n_modules_true} functional groups")

In [ ]:
# Quick look at the calcium activity
fig, ax = plt.subplots(figsize=(14, 4))
ax.imshow(exp.calcium.data, aspect='auto', cmap='hot', interpolation='none')
ax.set_xlabel('Frame')
ax.set_ylabel('Neuron')
ax.set_title(f'Calcium traces ({exp.n_cells} neurons, {exp.n_frames} frames)')
plt.colorbar(ax.images[0], ax=ax, fraction=0.02)
plt.tight_layout()
plt.show()

In [ ]:
print("Computing cell-cell functional connectivity")

sim_mat, sig_mat, pval_mat, cells_list, info = compute_cell_cell_significance(
    exp,
    data_type="calcium",
    ds=5,                      # Downsample by 5x for speed (~5x faster)
    n_shuffles_stage1=100,     # Stage 1 screening
    n_shuffles_stage2=10000,   # FFT makes high shuffle counts fast!
    pval_thr=0.001,
    multicomp_correction="holm",
    verbose=True
)

print(f"\nAnalysis complete!")
print(f"Total neuron pairs: {len(cells_list) * (len(cells_list)-1) / 2:.0f}")
print(f"Significant connections: {np.sum(sig_mat) / 2:.0f}")  # Divide by 2 for symmetry
print(f"Connection density: {np.sum(sig_mat) / (len(cells_list) * (len(cells_list)-1)):.3f}")

In [ ]:
print("Creating functional network")

sig_sparse = sp.csr_matrix(sig_mat)
net_binary = Network(
    adj=sig_sparse,
    preprocessing="giant_cc",
    name="Neural Functional Network (Binary)"
)

# Create weighted network (similarity values for significant edges only)
weighted_mat = sim_mat * sig_mat  # Zero out non-significant
weighted_sparse = sp.csr_matrix(weighted_mat)
net_weighted = Network(
    adj=weighted_sparse,
    preprocessing="giant_cc",
    name="Neural Functional Network (Weighted)"
)

print(f"Binary network: {net_binary.n} nodes, {net_binary.graph.number_of_edges()} edges")
print(f"Weighted network: {net_weighted.n} nodes, {net_weighted.graph.number_of_edges()} edges")

## 1. Network structure

Binary and weighted network properties: degree distribution,
connected components, clustering, module detection via Louvain
community detection, and comparison against a degree-preserving
null model.

In [ ]:
print("Network properties analysis")

# Basic properties
net = net_weighted
print(f"Network type: {'Directed' if net.directed else 'Undirected'}, "
      f"{'Weighted' if net.weighted else 'Binary'}")
print(f"Number of nodes: {net.n}")
print(f"Number of edges: {net.graph.number_of_edges()}")

# Degree statistics
degrees = [d for n, d in net.graph.degree()]
print(f"Average degree: {np.mean(degrees):.2f} +- {np.std(degrees):.2f}")
print(f"Max degree: {np.max(degrees)}")

# Clustering coefficient
if not net.directed:
    clustering = nx.average_clustering(net.graph)
    print(f"Average clustering coefficient: {clustering:.3f}")

# Connected components
if net.directed:
    n_weak = nx.number_weakly_connected_components(net.graph)
    n_strong = nx.number_strongly_connected_components(net.graph)
    print(f"Weakly connected components: {n_weak}")
    print(f"Strongly connected components: {n_strong}")
else:
    n_cc = nx.number_connected_components(net.graph)
    print(f"Connected components: {n_cc}")

# Spectral properties
print("\nSpectral analysis:")
adj_spectrum = net.get_spectrum("adj")
print(f"Largest eigenvalue (spectral radius): {np.max(np.abs(adj_spectrum)):.3f}")

if not net.directed:
    lap_spectrum = net.get_spectrum("lap")
    # Second smallest eigenvalue (algebraic connectivity)
    sorted_lap = np.sort(np.real(lap_spectrum))
    if len(sorted_lap) > 1:
        print(f"Algebraic connectivity: {sorted_lap[1]:.3f}")

In [ ]:
# Weighted network: weight distribution
if net.weighted:
    edges = net.graph.edges(data=True)
    weights = [d["weight"] for _, _, d in edges]
    print(f"Weight statistics:")
    print(f"  Mean weight: {np.mean(weights):.4f}")
    print(f"  Std weight:  {np.std(weights):.4f}")
    print(f"  Min weight:  {np.min(weights):.4f}")
    print(f"  Max weight:  {np.max(weights):.4f}")

In [ ]:
print("Detecting functional modules")

# Use Louvain community detection
# For weighted network, use weight attribute
if net.weighted:
    communities = nx_comm.louvain_communities(net.graph, weight="weight", seed=42)
else:
    communities = nx_comm.louvain_communities(net.graph, seed=42)

print(f"Found {len(communities)} functional modules:")
for i, community in enumerate(communities):
    print(f"  Module {i+1}: {len(community)} neurons")

# Create module assignment dictionary
module_assignment = {}
for module_idx, community in enumerate(communities):
    for node in community:
        module_assignment[node] = module_idx

In [ ]:
print("Null model comparison (degree-preserving randomization)")

n_replicates = 10

# Real network properties
real_clustering = nx.average_clustering(net.graph)
real_modularity = nx_comm.modularity(net.graph, communities)
lap_spectrum = net.get_spectrum("lap")
real_connectivity = np.sort(np.real(lap_spectrum))[1]

# Generate random networks and measure properties
null_clustering = []
null_modularity = []
null_connectivity = []

for i in range(n_replicates):
    rand_net = net.randomize(rmode="adj_iom")
    null_clustering.append(nx.average_clustering(rand_net.graph))
    rand_comms = nx_comm.louvain_communities(rand_net.graph, seed=i)
    null_modularity.append(nx_comm.modularity(rand_net.graph, rand_comms))
    rand_lap = rand_net.get_spectrum("lap")
    null_connectivity.append(np.sort(np.real(rand_lap))[1])

# Print comparison table
print(f"\n  {'Metric':<25s} {'Real':>8s} {'Null (mean +/- std)':>22s}")
print(f"  {'-' * 57}")

rows = [
    ("Clustering coefficient", real_clustering, null_clustering),
    ("Modularity", real_modularity, null_modularity),
    ("Algebraic connectivity", real_connectivity, null_connectivity),
]
for name, real_val, null_vals in rows:
    null_mean = np.mean(null_vals)
    null_std = np.std(null_vals)
    print(f"  {name:<25s} {real_val:>8.3f} {null_mean:>10.3f} +/- {null_std:.3f}")

print(f"\n  Replicates: {n_replicates}")

In [ ]:
# Visualization: adjacency matrix, degree distribution, network graph
fig = plt.figure(figsize=(16, 12))

# Get unique modules and create consistent color mapping
unique_modules = sorted(set(module_assignment.values()))
n_modules = len(unique_modules)
module_colors = plt.cm.tab10(np.linspace(0, 1, n_modules))
module_to_color = {mod: module_colors[i] for i, mod in enumerate(unique_modules)}

# 1. Similarity matrix
ax1 = plt.subplot(2, 3, 1)
im1 = ax1.imshow(sim_mat, cmap="hot", aspect="auto")
ax1.set_title("Similarity matrix (MI)", fontsize=12)
ax1.set_xlabel("Neuron ID")
ax1.set_ylabel("Neuron ID")
plt.colorbar(im1, ax=ax1, fraction=0.046)

# Add module boundaries if known (for 30/30/30/10/10/10 structure)
if module_sizes_true is not None:
    cumsum = np.cumsum([0] + module_sizes_true)
    for boundary in cumsum[1:-1]:
        ax1.axhline(boundary - 0.5, color="cyan", linewidth=2)
        ax1.axvline(boundary - 0.5, color="cyan", linewidth=2)

# 2. Significance matrix
ax2 = plt.subplot(2, 3, 2)
ax2.imshow(sig_mat, cmap="RdBu_r", aspect="auto", vmin=0, vmax=1)
ax2.set_title("Significance matrix", fontsize=12)
ax2.set_xlabel("Neuron ID")
ax2.set_ylabel("Neuron ID")

# 3. Network visualization with modules using spring layout
ax3 = plt.subplot(2, 3, 3)

# Use spring layout with parameters optimized for clustering
pos = nx.spring_layout(
    net.graph,
    k=1.5/np.sqrt(len(net.graph)),  # Optimal distance
    iterations=100,  # More iterations for better convergence
    seed=42
)

# Draw nodes colored by detected module
node_colors = [module_to_color[module_assignment[node]] for node in net.graph.nodes()]
nx.draw_networkx_nodes(
    net.graph, pos,
    node_color=node_colors,
    node_size=20,  # Smaller for 300 nodes
    ax=ax3,
    alpha=0.8
)

# Draw edges
if net.weighted:
    edges = net.graph.edges()
    weights = [net.graph[u][v]["weight"] for u, v in edges]
    nx.draw_networkx_edges(net.graph, pos, alpha=0.1, width=weights, ax=ax3)
else:
    nx.draw_networkx_edges(net.graph, pos, alpha=0.1, ax=ax3)

ax3.set_title("Functional network modules", fontsize=12)
ax3.axis("off")

# 4. Degree distribution
ax4 = plt.subplot(2, 3, 4)
degrees = [d for n, d in net.graph.degree()]
ax4.hist(degrees, bins=20, alpha=0.7, color="blue", edgecolor="black")
ax4.set_xlabel("Degree")
ax4.set_ylabel("Count")
ax4.set_title("Degree distribution", fontsize=12)
ax4.grid(True, alpha=0.3)

# 5. Module size distribution
ax5 = plt.subplot(2, 3, 5)
detected_sizes = {}
for node, module in module_assignment.items():
    detected_sizes[module] = detected_sizes.get(module, 0) + 1

modules = sorted(detected_sizes.keys())
sizes = [detected_sizes[m] for m in modules]
colors = [module_to_color[m] for m in modules]

ax5.bar(modules, sizes, color=colors)
ax5.set_xlabel("Module ID")
ax5.set_ylabel("Number of neurons")
ax5.set_title("Module sizes", fontsize=12)
ax5.grid(True, alpha=0.3, axis="y")

# 6. Similarity distribution
ax6 = plt.subplot(2, 3, 6)
# Get upper triangle p-values (excluding diagonal)
triu_indices = np.triu_indices_from(sig_mat, k=1)
pvals_upper = sim_mat[triu_indices]

ax6.hist(pvals_upper, bins=50, alpha=0.7, color="green", edgecolor="black")
ax6.set_xlabel("Similarity (MI)")
ax6.set_ylabel("Count")
ax6.set_title("Similarity distribution", fontsize=12)
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Save and load

Demonstrate sparse save/load roundtrip for the network adjacency,
similarity, and p-value matrices.

In [ ]:
# Save/load sparse roundtrip
def save_network(filename, sim_mat, sig_mat, pval_mat, cells, info):
    """
    Save network as sparse adjacency matrix with essential metadata only.

    Saves only:
    - Sparse adjacency matrix (significant connections)
    - Similarity/p-values for significant edges only
    - Cell IDs and scalar metadata
    - Excludes large shuffle arrays (random_shifts, me_total)
    """
    # Get indices of significant connections
    sig_indices = np.where(sig_mat > 0)

    # Extract only significant edges (sparse format)
    sparse_data = {
        "sig_mat_data": sig_mat[sig_indices],  # Binary significance
        "sim_values": sim_mat[sig_indices],    # Similarity for significant edges
        "pval_values": pval_mat[sig_indices],  # P-values for significant edges
        "sig_indices_i": sig_indices[0],       # Row indices
        "sig_indices_j": sig_indices[1],       # Column indices
        "matrix_shape": np.array(sig_mat.shape),  # Shape for reconstruction
        "cells": np.array(cells),
    }

    # Add optimal delays if available (only for significant edges)
    if "optimal_delays" in info:
        sparse_data["optimal_delays"] = info["optimal_delays"][sig_indices]

    # Add only scalar metadata (exclude huge shuffle arrays!)
    exclude_keys = {"random_shifts1", "me_total1", "random_shifts2", "me_total2"}
    for k, v in info.items():
        if k not in exclude_keys and isinstance(v, (int, float, str, np.integer, np.floating)):
            sparse_data[f"info_{k}"] = v

    # Save as compressed NPZ
    filename = filename if filename.endswith(".npz") else f"{filename}.npz"
    np.savez_compressed(filename, **sparse_data)

    # Report size savings
    n_sig = len(sig_indices[0])
    n_total = sig_mat.shape[0] * sig_mat.shape[1]
    density = n_sig / n_total
    print(f"Network saved to {filename}")
    print(f"  Significant edges: {n_sig}/{n_total} ({density*100:.2f}% density)")
    print(f"  Sparse storage: ~{n_sig * 32 / 1024:.1f} KB (vs {n_total * 8 / 1024:.1f} KB full matrix)")


def load_network(filename):
    """
    Load network from sparse format.
    """
    if not filename.endswith(".npz"):
        filename = f"{filename}.npz"

    data = np.load(filename, allow_pickle=True)

    # Load sparse format
    shape = tuple(data["matrix_shape"])
    sig_indices_i = data["sig_indices_i"]
    sig_indices_j = data["sig_indices_j"]

    # Initialize full matrices
    sig_mat_loaded = np.zeros(shape)
    sim_mat_loaded = np.zeros(shape)
    pval_mat_loaded = np.ones(shape)  # Initialize with 1.0 (non-significant)

    # Fill in significant edges
    sig_mat_loaded[sig_indices_i, sig_indices_j] = data["sig_mat_data"]
    sim_mat_loaded[sig_indices_i, sig_indices_j] = data["sim_values"]
    pval_mat_loaded[sig_indices_i, sig_indices_j] = data["pval_values"]

    cells_loaded = data["cells"].tolist()

    # Reconstruct info dict (with optimal delays if present)
    info_loaded = {}
    if "optimal_delays" in data:
        info_loaded["optimal_delays"] = np.zeros(shape)
        info_loaded["optimal_delays"][sig_indices_i, sig_indices_j] = data["optimal_delays"]

    # Add scalar metadata
    for k, v in data.items():
        if k.startswith("info_"):
            info_loaded[k.replace("info_", "")] = v

    print(f"Network loaded from {filename}")
    print(f"  Neurons: {len(cells_loaded)}")
    print(f"  Significant connections: {len(sig_indices_i) // 2:.0f} (undirected)")

    return sim_mat_loaded, sig_mat_loaded, pval_mat_loaded, cells_loaded, info_loaded


# Roundtrip demo using a temporary file
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = tmpdir + "/test_network"
    save_network(tmp_path, sim_mat, sig_mat, pval_mat, cells_list, info)

    sim_rt, sig_rt, pval_rt, cells_rt, info_rt = load_network(tmp_path)

    # Verify roundtrip
    assert np.allclose(sim_mat * sig_mat, sim_rt * sig_rt), "Similarity mismatch!"
    assert np.array_equal(sig_mat, sig_rt), "Significance mismatch!"
    print("\nRoundtrip verification: OK")

> **Cross-reference:** The spectral analysis below applies to *any*
> [`Network`](https://driada.readthedocs.io/en/latest/api/network/core.html#driada.network.net_base.Network) object -- not just cell-cell functional networks. Graph-based
> DR methods (Isomap, LLE, Laplacian Eigenmaps) construct a
> `ProximityGraph` that inherits from `Network`, so all the spectral,
> entropy, and community analysis shown below works on DR graphs too.
> See the "Graph structure behind DR" subsection of
> [Notebook 03](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/03_population_geometry_dr.ipynb)
> for a demonstration.

## 2. Spectral analysis

Eigendecomposition of the adjacency and normalized Laplacian matrices
reveals global structure that is invisible to local metrics.

**Adjacency spectrum.** The adjacency eigenvalues reflect community
structure: isolated clusters produce near-degenerate eigenvalue groups.
The spectral radius (largest $|\lambda|$) scales with the mean degree
for random graphs.

**Normalized Laplacian.**
$L_{\text{norm}} = I - D^{-1/2} A D^{-1/2}$
has eigenvalues bounded in $[0, 2]$ regardless of network size or degree
distribution, making it suitable for cross-network comparison.

In [ ]:
net_spectral = net_binary

print("Eigendecomposition")

adj_spectrum = net_spectral.get_spectrum("adj")
print(f"\nAdjacency matrix:")
print(f"  Spectral radius (max |lambda|): {np.max(np.abs(adj_spectrum)):.3f}")
print(f"  Min eigenvalue: {np.min(np.real(adj_spectrum)):.3f}")
print(f"  Max eigenvalue: {np.max(np.real(adj_spectrum)):.3f}")

nlap_spectrum = net_spectral.get_spectrum("nlap")
sorted_nlap = np.sort(np.real(nlap_spectrum))

# Each zero eigenvalue corresponds to one connected component.
# Since we extracted the giant connected component, expect exactly 1.
n_zero = int(np.sum(np.abs(sorted_nlap) < 1e-6))
n_components = n_zero

print(f"\nNormalized Laplacian (L = I - D^(-1/2) A D^(-1/2)):")
print(f"  Eigenvalue range: [0, 2]")
print(f"  Smallest eigenvalue: {sorted_nlap[0]:.6f}")
print(f"  Zero eigenvalues: {n_zero}  (= number of connected components)")
if n_components == 1:
    print(f"  Graph is connected (single component)")
else:
    print(f"  WARNING: graph has {n_components} disconnected components")

# The Fiedler value (first non-zero eigenvalue) is the algebraic
# connectivity of the normalized Laplacian. Larger values indicate
# a graph that is harder to disconnect by removing edges.
if len(sorted_nlap) > n_zero:
    fiedler = sorted_nlap[n_zero]
    print(f"  Fiedler value (lambda_{n_zero + 1}): {fiedler:.4f}")
    print(f"    (Algebraic connectivity -- larger = harder to disconnect)")
print(f"  Spectral gap: {sorted_nlap[n_zero] - sorted_nlap[0]:.4f}")
print(f"  Largest eigenvalue: {sorted_nlap[-1]:.4f}")

adj_vecs = net_spectral.get_eigenvectors("adj")
print(f"\nAdjacency eigenvectors: {adj_vecs.shape}  (N x N matrix)")

### Spectral metrics

**IPR (Inverse Participation Ratio):**
$\text{IPR} = \sum_i |v_i|^4$ for each eigenvector $v$.
For a vector uniformly spread over $N$ nodes, $\text{IPR} = 1/N$.
For a vector concentrated on one node, $\text{IPR} = 1$.

In [ ]:
print("IPR analysis (eigenvector localization)")

ipr_adj = net_spectral.get_ipr("adj")
ipr_nlap = net_spectral.get_ipr("nlap")

delocalized_bound = 1.0 / net_spectral.n

print(f"\nAdjacency IPR:")
print(f"  Mean: {np.mean(ipr_adj):.4f}")
print(f"  Min:  {np.min(ipr_adj):.4f}  (delocalized bound 1/N = {delocalized_bound:.4f})")
print(f"  Max:  {np.max(ipr_adj):.4f}")

print(f"\nNormalized Laplacian IPR:")
print(f"  Mean: {np.mean(ipr_nlap):.4f}")
print(f"  Min:  {np.min(ipr_nlap):.4f}")
print(f"  Max:  {np.max(ipr_nlap):.4f}")

**Complex spacing ratios.**
For a symmetric (Hermitian) matrix all eigenvalues are real, so the
complex spacing ratios $z = (\lambda_{\text{nn}} - \lambda) /
(\lambda_{\text{nnn}} - \lambda)$ collapse to the real line and
$\arg(z)$ is either $0$ or $\pi$. `get_z_values` builds a
$k$-d tree in the complex plane and returns one ratio per unique
eigenvalue.

**Localization signatures.**
$\langle\cos(\arg z)\rangle$ measures phase coherence of spacing ratios;
$\langle 1/|z|^2 \rangle$ amplifies cases where nearest and next-nearest
neighbour distances differ strongly. Zero $z$-values (from degenerate
eigenvalues) are filtered internally to avoid singularities.

In [ ]:
print("Spacing ratios and localization signatures")

z_dict = net_spectral.get_z_values("nlap")

# Discard the z-value for the zero eigenvalue (one per connected component).
# It is trivial and would distort 1/|z|^2 statistics.
z_filtered = {k: v for k, v in z_dict.items() if np.abs(k) > 1e-6}
n_discarded = len(z_dict) - len(z_filtered)
z_list = np.array(list(z_filtered.values()))
z_mags = np.abs(z_list)

print(f"\n  Total eigenvalues with z-values: {len(z_dict)}")
print(f"  Discarded (zero eigenvalues): {n_discarded}")
print(f"  Used for analysis: {len(z_filtered)}")

print(f"\nComplex spacing ratios z = (lambda_nn - lambda) / (lambda_nnn - lambda):")
print(f"  Count: {len(z_list)}")
print(f"  Mean |z|: {np.mean(z_mags):.4f}  (bounded in [0, 1])")
print(f"  Std |z|:  {np.std(z_mags):.4f}")

In [ ]:
mean_inv_r2, mean_cos_phi = net_spectral.localization_signatures("nlap")
print(f"\nLocalization signatures (undirected, normalized Laplacian):")
print(f"  <cos(arg(z))>: {mean_cos_phi:.4f}")
print(f"  <1/|z|^2>:     {mean_inv_r2:.4f}")

**Communicability.**
Estrada communicability $\text{EE} = \sum_i \exp(\lambda_i)$ where
$\lambda_i$ are adjacency eigenvalues. This counts walks of all lengths,
weighted by $1/k!$.

**Bipartivity index.** Ratio of even-length to total weighted walks,
using both $\exp(\lambda)$ and $\exp(-\lambda)$ of the adjacency spectrum.

**Gromov hyperbolicity.** For every 4-point set, measures how far the
shortest-path metric deviates from a tree metric.
$\delta = 0$ means the network is a tree; larger values indicate cycles.

In [ ]:
print("Communicability and network geometry")

comm = net_spectral.calculate_estrada_communicability()
print(f"\nEstrada communicability index: {comm:.4g}")

bipartivity = net_spectral.get_estrada_bipartivity_index()
print(f"Estrada bipartivity index: {bipartivity:.4f}")
print(f"  (1.0 = perfectly bipartite, 0.0 = far from bipartite)")

hyp = net_spectral.calculate_gromov_hyperbolicity(num_samples=50000)
print(f"\nGromov hyperbolicity (mean delta): {hyp:.3f}")
print(f"  (0 = tree-like, higher = more cycle-rich)")

**Thermodynamic entropy.**
Temperature sweep: low $t$ probes local structure, high $t$ probes global.

- **Von Neumann entropy:** $S(t) = -\sum_i p_i \log_2 p_i$ where
  $p_i = \exp(-\lambda_i / t) / Z$ is the Boltzmann distribution over
  normalized Laplacian eigenvalues.
- **Free entropy:** $F(t) = \log_2 Z$ where $Z = \sum_i \exp(-\lambda_i / t)$.
- **Renyi $q$-entropy:** $S_q(t) = \log_2(\sum_i p_i^q) / (1 - q)$.
  At $q = 2$, this is related to the purity of the Gibbs state.

In [ ]:
print("Thermodynamic entropy analysis")

tlist = np.logspace(-2, 2, 50)

vn_entropy = net_spectral.calculate_thermodynamic_entropy(tlist, norm=True)
print(f"\nVon Neumann entropy S(t) [normalized Laplacian]:")
print(f"  At t=0.01: {vn_entropy[0]:.3f} bits")
print(f"  At t=1.00: {vn_entropy[len(tlist) // 2]:.3f} bits")
print(f"  At t=100:  {vn_entropy[-1]:.3f} bits")
print(f"  Max entropy: {np.max(vn_entropy):.3f} bits"
      f"  (upper bound = log2(N) = {np.log2(net_spectral.n):.2f})")

free_ent = net_spectral.calculate_free_entropy(tlist, norm=True)
print(f"\nFree entropy F(t) = log2(Z):")
print(f"  At t=0.01: {free_ent[0]:.3f}")
print(f"  At t=100:  {free_ent[-1]:.3f}")

q_ent = net_spectral.calculate_q_entropy(q=2, tlist=tlist, norm=True)
print(f"\nRenyi 2-entropy S_2(t):")
print(f"  At t=0.01: {q_ent[0]:.3f} bits")
print(f"  At t=100:  {q_ent[-1]:.3f} bits")

### Laplacian Eigenmaps embedding

LEM uses the normalized Laplacian and selects the smallest non-zero
eigenvectors as embedding coordinates. Nearby nodes in the graph map
to nearby points in the embedding.

In [ ]:
print("Laplacian Eigenmaps embedding")

dim = 3
net_spectral.construct_lem_embedding(dim)

# Access stored embedding (shape: dim x n_nodes)
if hasattr(net_spectral.lem_emb, "toarray"):
    emb_data = np.real(net_spectral.lem_emb.toarray())
else:
    emb_data = np.real(np.asarray(net_spectral.lem_emb))

print(f"\nLEM embedding ({dim}D):")
print(f"  Shape: {emb_data.shape}")
print(f"  Dim 1 range: [{emb_data[0].min():.3f}, {emb_data[0].max():.3f}]")
print(f"  Dim 2 range: [{emb_data[1].min():.3f}, {emb_data[1].max():.3f}]")
print(f"  Dim 3 range: [{emb_data[2].min():.3f}, {emb_data[2].max():.3f}]")

# Plot LEM embedding colored by detected module
fig, ax = plt.subplots(figsize=(8, 6))
# Map nodes in the spectral network to their modules
node_list = list(net_spectral.graph.nodes())
node_module_colors = [module_to_color.get(module_assignment.get(n, 0), "gray") for n in node_list]
ax.scatter(emb_data[0], emb_data[1], s=15, alpha=0.7, c=node_module_colors, edgecolors="none")
ax.set_xlabel("LEM dim 1")
ax.set_ylabel("LEM dim 2")
ax.set_title("Laplacian Eigenmaps (colored by module)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Directed variant

Randomly orienting edges breaks the Hermitian symmetry, giving complex
eigenvalues and $z$-values that spread across the complex plane. This
demonstrates the full capability of the complex spacing ratio framework.
In practice, directed networks arise from causal or effective
connectivity (e.g. Granger causality, transfer entropy).

In [ ]:
dir_adj = turn_to_partially_directed(net_spectral.adj, directed=1.0)
dir_net = Network(adj=dir_adj, preprocessing=None, name="Directed variant")

z_dict_dir = dir_net.get_z_values("adj")
z_list_dir = np.array(list(z_dict_dir.values()))
z_mags_dir = np.abs(z_list_dir)

print(f"\nDirected variant (randomly oriented edges):")
print(f"  Eigenvalues: {len(z_dict_dir)} (complex)")
print(f"  Mean |z|: {np.mean(z_mags_dir):.4f}")
print(f"  Std |z|:  {np.std(z_mags_dir):.4f}")

mean_inv_r2_d, mean_cos_phi_d = dir_net.localization_signatures("adj")
print(f"\nLocalization signatures (directed adjacency):")
print(f"  <cos(arg(z))>: {mean_cos_phi_d:.4f}")
print(f"  <1/|z|^2>:     {mean_inv_r2_d:.4f}")

# Plot complex eigenvalues
dir_spectrum = dir_net.get_spectrum("adj")
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(np.real(dir_spectrum), np.imag(dir_spectrum), s=20, alpha=0.6,
           c="steelblue", edgecolors="none")
theta = np.linspace(0, 2 * np.pi, 200)
r_max = np.max(np.abs(dir_spectrum))
ax.plot(r_max * np.cos(theta), r_max * np.sin(theta), "k--", alpha=0.3, linewidth=0.8)
ax.set_xlabel("Re(lambda)")
ax.set_ylabel("Im(lambda)")
ax.set_title("Directed network eigenvalues")
ax.set_aspect("equal")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("Spectral null model comparison (degree-preserving randomization)")

real_ipr = np.mean(net_spectral.get_ipr("nlap"))
nlap_sorted = np.sort(np.real(net_spectral.get_spectrum("nlap")))
real_fiedler = nlap_sorted[nlap_sorted > 1e-6][0]

n_replicates = 10
null_comm = []
null_bipart = []
null_ipr = []
null_fiedler = []

for _ in range(n_replicates):
    rand_net = net_spectral.randomize(rmode="adj_iom")
    null_comm.append(rand_net.calculate_estrada_communicability())
    null_bipart.append(rand_net.get_estrada_bipartivity_index())
    null_ipr.append(np.mean(rand_net.get_ipr("nlap")))
    rand_nlap = np.sort(np.real(rand_net.get_spectrum("nlap")))
    nonzero = rand_nlap[rand_nlap > 1e-6]
    null_fiedler.append(nonzero[0] if len(nonzero) > 0 else 0.0)

print(f"\n  {'Metric':<28s} {'Real':>12s} {'Null (mean +/- std)':>25s}")
print(f"  {'-' * 67}")

rows = [
    ("Communicability", comm, null_comm),
    ("Bipartivity", bipartivity, null_bipart),
    ("Mean nlap IPR", real_ipr, null_ipr),
    ("Fiedler value", real_fiedler, null_fiedler),
]
for name, real_val, null_vals in rows:
    null_mean = np.mean(null_vals)
    null_std = np.std(null_vals)
    print(f"  {name:<28s} {real_val:>12.4g} {null_mean:>12.4g} +/- {null_std:.4g}")

print(f"\n  Replicates: {n_replicates}")

In [ ]:
# 2x3 spectral summary figure
fig, axes = plt.subplots(2, 3, figsize=(18, 11))

# 1. Adjacency spectrum histogram
ax = axes[0, 0]
real_spec = np.real(adj_spectrum)
nbins = int(np.ceil(np.log2(len(real_spec)))) + 1
ax.hist(real_spec, bins=nbins, edgecolor="black", linewidth=0.5, alpha=0.8)
ax.set_xlabel("Eigenvalue")
ax.set_ylabel("Count")
ax.set_title("Adjacency spectrum")
ax.grid(True, alpha=0.3)

# 2. Normalized Laplacian IPR (sorted by magnitude)
ax = axes[0, 1]
ax.plot(np.sort(ipr_nlap), "o-", markersize=2, linewidth=0.8)
ax.axhline(1.0 / net_spectral.n, color="r", linestyle="--", label=f"1/N = {1.0/net_spectral.n:.4f}")
ax.set_xlabel("Eigenvector index (sorted)")
ax.set_ylabel("IPR")
ax.set_title("Normalized Laplacian IPR")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# 3. Thermodynamic entropy curve
ax = axes[0, 2]
ax.semilogx(tlist, vn_entropy, "b-", linewidth=2)
ax.set_xlabel("Temperature (t)")
ax.set_ylabel("Entropy (bits)")
ax.set_title("Von Neumann entropy S(t)")
ax.grid(True, alpha=0.3)

# 4. Complex spacing ratio density (directed network).
# Randomly orienting edges breaks the Hermitian symmetry, giving
# complex eigenvalues whose z-values spread across the unit disk.
# The density pattern is a fingerprint of the spectral universality class.
ax = axes[1, 0]
zr, zi = np.real(z_list_dir), np.imag(z_list_dir)
xy = np.vstack([zr, zi])
kde = gaussian_kde(xy, bw_method=0.25)
grid_n = 200
pad = 0.15
xmin, xmax = zr.min() - pad, zr.max() + pad
ymin, ymax = zi.min() - pad, zi.max() + pad
xg = np.linspace(xmin, xmax, grid_n)
yg = np.linspace(ymin, ymax, grid_n)
Xg, Yg = np.meshgrid(xg, yg)
Z = kde(np.vstack([Xg.ravel(), Yg.ravel()])).reshape(grid_n, grid_n)
ax.pcolormesh(Xg, Yg, Z, shading="auto", cmap="inferno")
# Unit circle for reference (|z| <= 1)
theta = np.linspace(0, 2 * np.pi, 200)
ax.plot(np.cos(theta), np.sin(theta), "w--", linewidth=0.8, alpha=0.5)
ax.set_xlabel("Re(z)")
ax.set_ylabel("Im(z)")
ax.set_title("Complex spacing ratios (directed)")
ax.set_aspect("equal")

# 5. LEM embedding (first 2 dims)
ax = axes[1, 1]
ax.scatter(emb_data[0], emb_data[1], s=15, alpha=0.7, c="steelblue", edgecolors="none")
ax.set_xlabel("LEM dim 1")
ax.set_ylabel("LEM dim 2")
ax.set_title("Laplacian Eigenmaps")
ax.grid(True, alpha=0.3)

# 6. Real vs null communicability
ax = axes[1, 2]
ax.hist(null_comm, bins=8, edgecolor="black", linewidth=0.5, alpha=0.6,
        color="gray", label="Null model")
ax.axvline(comm, color="red", linewidth=2, label=f"Real = {comm:.1f}")
ax.set_xlabel("Communicability")
ax.set_ylabel("Count")
ax.set_title("Real vs null model")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()